In [1]:
from spectacle.modeling.models import Absorption1D
from spectacle.core.spectra import Spectrum1D
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from spectacle.modeling.fitting import LevMarFitter
from uncertainties import unumpy as unp

from astropy.modeling import fitting

INFO:root:Added misty to custom loaders.
INFO:root:Added my-format to custom loaders.


In [2]:
%matplotlib notebook

In [3]:
# Load real data
with fits.open("/Users/nearl/projects/hst_proposal/QSOALS/3C066A/3C066A_coadd_FUVM_final_all.fits") as hdulist:
    disp, flux, uncert, dq = hdulist[1].data['WAVE'], hdulist[1].data['FLUX'], \
                             hdulist[1].data['ERROR'], hdulist[1].data['DQ']

In [4]:
# Create a mask and apply it since we don't really care about the entire spectrum
# mask = [(disp > 1276) & (disp < 1283)]
mask = [(disp > 1252) & (disp < 1256)]

flux = flux[mask]
disp = disp[mask]
uncert = uncert[mask]

# Currently, the fitting assumes that the data is normalized to 1.0
tmp_flux = unp.uarray(flux, uncert)
tmp_flux /= np.median(tmp_flux)
flux, uncert = unp.nominal_values(tmp_flux), unp.std_devs(tmp_flux)

# Create spectrum object
spectrum = Spectrum1D(flux, dispersion=disp, uncertainty=uncert)

In [7]:
mod = Absorption1D(slope=0.1, intercept=3.0)
mod.add_line(5, 10**4)

print(mod.parameters)

fit = fitting.LevMarLSQFitter()

new_mod = fit(mod, np.arange(100), np.ones(100))

[ 0.1  3. ]
() {}
() {}
() {}
() {}
() {}
() {}
() {}
() {}
() {}
() {}
